# K-nearest neighbours model 😮

## /////////////////////////////////////////////

### Issues With Last Time
-Code was incredibly ugly and unelegant <br />
-There was also experimental code in file (Made it hard to read) <br />
-No stratified sampling <br />
-A bit too good of a model (Seemed suspicious)

In [83]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")

<br />



## <span style="color: red;"> STEP 1) Insert the data and split into features and labels and encode</span>

In [85]:
dfWDBC = pd.read_csv("wdbc/wdbc.data", header = None)
#add columns
colHead = ["ID", "B/M", "Radius","Texture",
           "Perimeter","Area","Smoothness","Compactness","Concavity","ConcavePoints","Symmetry","FractalDimension",
          "RadiusSE","TextureSE","PerimeterSE","AreaSE","SmoothnessSE","CompactnessSE","ConcavitySE",
           "ConcavePointsSE","SymmetrySE","FractalDimensionSE","RadiusWorst","TextureWorst",
           "PerimeterWorst","AreaWorst","SmoothnessWorst","CompactnessWorst","ConcavityWorst",
           "ConcavePointsWorst","SymmetryWorst","FractalDimensionWorst"]
dfWDBC.columns = colHead


#split into test and features
features = dfWDBC[["Texture","Symmetry"]]
label = dfWDBC["B/M"]

#Need to encode B/M before proceeding
le = preprocessing.LabelEncoder()
label = le.fit_transform(label)
label = pd.DataFrame(data = label, columns = ["B/M"])


#Need to encode the features
ss = StandardScaler()
features = ss.fit_transform(features)

<br />


##  <span style="color: red;"> STEP 2) split the features and labels into appropriate test and training data using stratified sampling </span>

In [86]:
#All nice and stratified :)
x_train, x_test, y_train, y_test = train_test_split(features, label, test_size=0.2, stratify = label["B/M"].values, random_state= 1)
#Simples

</br>

##  <span style="color: red;"> STEP 3) Produce a k-nearest neighbours model and test the accuracy </span>

In [87]:
k = 10
model = KNeighborsClassifier(n_neighbors=k)
model.fit(x_train,y_train)

print("Test Size = " + str(len(y_test)))
#Testing Accuracy
y_pred = model.predict(x_test)
print("Accuracy Score = " + str(metrics.accuracy_score(y_test,y_pred)))

#Testing errors using accuracy matrix (Thanks Nicky)
c = confusion_matrix(y_test, y_pred)
print(f'True negatives: {c[0][0]}')
print(f'False negatives: {c[1][0]}')
print(f'True positives: {c[1][1]}')
print(f'False positives: {c[0][1]}')

Test Size = 114
Accuracy Score = 0.7368421052631579
True negatives: 62
False negatives: 20
True positives: 22
False positives: 10


////// Left out pretty graph i made last time because its a pain to make and not neccessary rn ////////

# <span style="color: purple;"> My Dilema 😮 </span>

So when i run the k nearest neighbours model with the 2 features being radius and texture it comes up with a bonkers accuracy of around 90%. If you look at the graph though the classification looks so simple that you could probably get a similar result using pen and paper. is this a good thing or should we use the k nearest neighbours for a more complex classification task 🤨

<img src = "https://64.media.tumblr.com/7f8b2bcc968b6662755baf7c3508cf73/tumblr_nshp2dKcyr1utmzuco1_500.gifv">

## Your opinions guys???? 🤔

# Testing Git lol